In [73]:
import pandas as pd
from ydata_profiling import ProfileReport
from category_encoders.cat_boost import CatBoostEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../data/client_attrition_train.csv')

In [4]:
df

,customer_id,customer_age,customer_sex,customer_number_of_dependents,customer_education,customer_civil_status,customer_salary_range,customer_relationship_length,customer_available_credit_limit,credit_card_classification,...,period_inactive,contacts_in_last_year,credit_card_debt_balance,remaining_credit_limit,transaction_amount_ratio,total_transaction_amount,total_transaction_count,transaction_count_ratio,average_utilization,account_status
0,755410,38.0,F,2,High School,Married,40-60K,31,1593.0,Blue,...,2,4,1091,502.0,0.870,4136.0,67,0.718,0.685,open
1,568093,46.0,F,2,Graduate,Unknown,below 40K,40,6568.0,Blue,...,2,2,0,6568.0,0.101,1507.0,33,0.222,0.000,closed
2,595389,43.0,M,1,High School,Married,80-120K,30,34516.0,Silver,...,1,3,2045,32471.0,0.590,4081.0,54,0.421,0.059,open
3,287252,46.0,F,4,High School,Married,below 40K,36,2374.0,Blue,...,2,1,1332,1042.0,0.686,4253.0,81,0.884,0.561,open
4,231901,40.0,M,4,High School,Single,80-120K,29,12978.0,Blue,...,3,2,0,12978.0,0.628,14134.0,85,0.700,0.000,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,435491,NaN,F,2,High School,Single,40-60K,36,1677.0,Blue,...,3,1,0,1677.0,0.700,4035.0,87,0.891,0.000,open
10123,136052,46.0,F,4,Uneducated,Single,Unknown,33,224541.0,Blue,...,3,3,1749,23200.0,0.626,4092.0,77,0.540,0.070,open
10124,336583,44.0,M,0,High School,Single,60-80K,36,6606.0,Blue,...,3,2,2517,4089.0,0.825,4493.0,68,0.659,0.381,open
10125,817129,39.0,F,5,Graduate,Single,below 40K,28,2077.0,Blue,...,3,2,0,2077.0,0.590,3647.0,69,0.917,0.000,open


In [12]:
profile = ProfileReport(df, title="Profiling Report")

In [14]:
profile.to_file("clasification_profile_report.html")

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

EDA Conclusions/Tasks:
Columns:
- Customer ID - to be dropped
- Customer Age - needs to be imputed (maybe check relationship length for hints)
- Customer Sex - encode
- Customer Education - encode
- Customer Civil Status - encode
- Customer Salary Range - encode, impute
- Customer Relationship Length - a lot of 36 year long relationships - to be further inspected
- Customer Available Credit Limit - use log
- Credit Card Classification - encode
- Total Products - use log? create bins for 0-5, 5-10, 10 or more products
- Period Inactive - create bins for 0-3 and 3 or more
- Contacts in last year - no additional ideas atm
- Credit Card Debt Balance - create bin for cc bal = 0, standardize
- Remaining Credit Card Limit - use log
- Transaction Amount Ratio - additional bins for 0-0.5, 0.5-1, 1-2, 2 or more?
- Total Transaction Amount - create bin for transaction amt = 0, use log
- Total Transaction Count - create bins for 0-25, 25-50, 50-100, 100 or more
- Transaction Count Ratio - additional bins for 0-0.5, 0.5-1, 1-2, 2 or more?
- Average Utilization - additional bins for 0-0.2, 0.2-0.6, 0.6 or more.
- Account status - encode


Imputting:
- check relation between customer age and customer relationship length
- check relation between salary range and sex (and maybe cc limit)

## Encoding

In [31]:
cb_encoder = CatBoostEncoder()
cols_to_be_encoded = ['customer_age', 'customer_sex', 'customer_education', 'customer_civil_status', 'customer_salary_range', 'credit_card_classification']
target = df.account_status.map({'open': 0, 'closed': 1})
features = df.drop(columns=['account_status'])

In [32]:
cb_encoder.fit(X=features, y=target, cols=cols_to_be_encoded, handle_missing='return_nan')

CatBoostEncoder(cols=['customer_sex', 'customer_education',
                      'customer_civil_status', 'customer_salary_range',
                      'credit_card_classification'])

In [35]:
cb_encoder.transform(df.drop(columns=['account_status']))

customer_id                          int64
customer_age                       float64
customer_sex                       float64
customer_number_of_dependents        int64
customer_education                 float64
customer_civil_status              float64
customer_salary_range              float64
customer_relationship_length         int64
customer_available_credit_limit    float64
credit_card_classification         float64
total_products                       int64
period_inactive                      int64
contacts_in_last_year                int64
credit_card_debt_balance             int64
remaining_credit_limit             float64
transaction_amount_ratio           float64
total_transaction_amount           float64
total_transaction_count              int64
transaction_count_ratio            float64
average_utilization                float64
dtype: object

## Transformations

In [39]:
df['customer_available_credit_limit_below_1450'] = np.where(df['customer_available_credit_limit'] < 1450, 1, 0)
df['customer_available_credit_limit'] = np.log(df['customer_available_credit_limit'])

In [40]:
df['total_products_0_5'] = np.where(df['total_products'] <= 5, 1, 0)
df['total_products_6_10'] = np.where((df['total_products'] > 5) & (df['total_products'] <= 10), 1 ,0)
df['total_products_more_than_10'] = np.where(df['total_products'] > 10, 1, 0)

In [41]:
df['period_inactive_0_3'] = np.where(df['period_inactive'] <= 3, 1, 0)

In [43]:
df['credit_card_debt_balance_0'] = np.where(df['credit_card_debt_balance'] == 0, 1, 0)
# do scaling inside CV

In [44]:
df['remaining_credit_limit'] = np.log(df['remaining_credit_limit'])

In [45]:
df['transaction_amount_ratio_0_05'] = np.where(df['transaction_amount_ratio'] <= 0.5, 1 ,0)
df['transaction_amount_ratio_05_1'] = np.where((df['transaction_amount_ratio'] > 0.5) & (df['transaction_amount_ratio'] <= 1), 1, 0)
df['transaction_amount_ratio_1_2'] = np.where((df['transaction_amount_ratio'] > 1) & (df['transaction_amount_ratio'] <= 2), 1, 0)
df['transaction_amount_ratio_more_than_2'] = np.where(df['transaction_amount_ratio'] > 2, 1, 0)

In [48]:
df['total_transaction_amount'] = np.log(df['total_transaction_amount'])

In [49]:
df['total_transaction_count_0_25'] = np.where(df['total_transaction_count'] <= 25, 1, 0)
df['total_transaction_count_26_50'] = np.where((df['total_transaction_count'] > 25) & (df['total_transaction_count'] <= 50), 1, 0)
df['total_transaction_count_51_100'] = np.where((df['total_transaction_count'] > 50) & (df['total_transaction_count'] <= 100), 1, 0)
df['total_transaction_count_more_than_100'] = np.where(df['total_transaction_count'] > 100, 1, 0)

In [50]:
df['transaction_count_ratio_0_05'] = np.where(df['transaction_count_ratio'] <= 0.5, 1 ,0)
df['transaction_count_ratio_05_1'] = np.where((df['transaction_count_ratio'] > 0.5) & (df['transaction_count_ratio'] <= 1), 1, 0)
df['transaction_count_ratio_1_2'] = np.where((df['transaction_count_ratio'] > 1) & (df['transaction_count_ratio'] <= 2), 1, 0)
df['transaction_count_ratio_more_than_2'] = np.where(df['transaction_count_ratio'] > 2, 1, 0)

In [51]:
df['average_utilization_0_02'] = np.where(df['average_utilization'] <= 0.2, 1, 0)
df['average_utilization_02_06'] = np.where((df['average_utilization'] > 0.2) & (df['average_utilization'] <= 0.6), 1, 0)
df['average_utilization_more_than_06'] = np.where(df['average_utilization'] > 0.6, 1, 0)

In [84]:
# here, where utilization is almost 0
sns.histplot(df['total_transaction_amount'])
sns.histplot(df[df['average_utilization'] == 0]['total_transaction_amount'])

<AxesSubplot:xlabel='total_transaction_amount', ylabel='Count'>